# Predict multiple occupancy from motion sensors

## Setup

In [1]:
# Imports
import sqlite3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
# Load tables as pandas dataframes 
db = '../data/data.db'

try:
    conn = sqlite3.connect(db)
    
except sqlite3.Error as error:
    print('Error occurred - ', error)

df_homes = pd.read_sql_query("select * from homes", conn)
df_motion = pd.read_sql_query("select * from motion", conn)

## Homes table data validation

In [3]:
df_homes

,id,multiple_occupancy
0,dc922bd9bf8a2a9437642fe0dce9dcaa,0
1,d40fc88629271da7b5833ed3f3222d17,1
2,3fb064da180f02862063eca8e3b609c0,1
3,16d71b9c46d9abd765bf395818efe527,0
4,458264665736fd26184a8ea5641e6c48,1
...,...,...
101,dbe580ed60d7c329931d98a0e2ad0ea3,1
102,04a16138cd7e0fa768858ff7c02856d8,1
103,62c8145530aca30cb0455dc386c0a9d6,1
104,3a5cc5ac41ab3b18b63d6d8366e30fb0,1


In [4]:
df_homes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  106 non-null    object
 1   multiple_occupancy  106 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 1.8+ KB


In [5]:
df_homes.id = df_homes.id.astype('string')

In [6]:
df_homes.dtypes

id                    string[python]
multiple_occupancy             int64
dtype: object

In [7]:
df_homes.nunique()

id                    106
multiple_occupancy      2
dtype: int64

In [8]:
df_homes.isnull().sum()

id                    0
multiple_occupancy    0
dtype: int64

In [9]:
df_homes.multiple_occupancy.value_counts()

multiple_occupancy
1    61
0    45
Name: count, dtype: int64

In [10]:
df_homes = df_homes.rename(columns={'id':'home_id'})
df_homes.dtypes

home_id               string[python]
multiple_occupancy             int64
dtype: object

## Motion table data validation


In [11]:
df_motion

,id,home_id,datetime,location
0,e41218b439d933a1cd9ad158f78e9198,205c42ec747e2db13cb92087a99433f1,2024-01-01 00:00:10+00,lounge
1,92d48d869ae50b0764cfb8d70494f618,7d2f2e0a9e059b4fb8106bb0ad4b8a39,2024-01-01 00:00:17+00,lounge
2,65c18ba64884442dd47c2fd4cf3630e4,44a880cc6fc3a7db3464092f650ae7f1,2024-01-01 00:00:18+00,lounge
3,90d6336d189c929aa50fa08e5aee5f41,49b83fce41b676266b98cd1e095f1c11,2024-01-01 00:00:43+00,lounge
4,6e3d73bed24b95ffdfe5ec017787f039,14328a0b7574e912c2e23d62c9476a07,2024-01-01 00:00:57+00,lounge
...,...,...,...,...
580312,5ec75a93dcc136e6a4c7a4483d60b2ed,3a1f53e2e242bff03301a250135f894b,2024-01-31 23:58:26+00,bathroom1
580313,406c70038a540d29b1e68250d5ed341c,37b99328702f9fdc68c431a4ac450f2e,2024-01-31 23:58:36+00,kitchen
580314,5168c52c3c022d747c81a6e6ce868e3f,3a1f53e2e242bff03301a250135f894b,2024-01-31 23:59:02+00,bedroom1
580315,40cad449e7cef4164b02672e8caff379,2b5ce37a65e82735416d69b987d99fe8,2024-01-31 23:59:09+00,hallway


In [12]:
df_motion.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580317 entries, 0 to 580316
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        580317 non-null  object
 1   home_id   580317 non-null  object
 2   datetime  580317 non-null  object
 3   location  580317 non-null  object
dtypes: object(4)
memory usage: 17.7+ MB


In [13]:
df_motion.nunique()

id          580317
home_id         50
datetime    502280
location        10
dtype: int64

In [14]:
df_motion.isnull().sum()

id          0
home_id     0
datetime    0
location    0
dtype: int64

In [15]:
df_motion.home_id.value_counts()

home_id
3a1f53e2e242bff03301a250135f894b    26472
c026d3469f489d341ab145bd62d00796    22658
2739e3f7409068a94cf6e3eac643c2e7    20501
49b83fce41b676266b98cd1e095f1c11    20019
83740365903618c9bec6c786ce5643e9    18250
b2791887c7c0dca72cc0c9f921a976df    18078
3b57a456d431b64da3485c6aedfc13c1    17372
424fc251603608ed5de614b11ce0c409    17137
44a880cc6fc3a7db3464092f650ae7f1    16375
c07573c70bf978b70906b09edcbc6d57    16282
205c42ec747e2db13cb92087a99433f1    16191
14328a0b7574e912c2e23d62c9476a07    15977
352d5124d21e0cd52ea4124ada6391c6    15508
53dcb3b61f7eec0a28d5983c9b6b29db    15418
0f44ff9edd221e417195f4398d2f3853    15303
930874c7cb4c0a68e876a6b5b044d22a    14946
62c8145530aca30cb0455dc386c0a9d6    14649
5ce0757be7d84242c9c73b975e93146b    13700
6be9c116a8a5e94d01efe4b6ebb3a1fe    13351
a8dfbc0f5d4c2e5601b5c9cd870a6de7    12654
e081c6aaaedac3d3678d3307388026fb    11846
a3b30fd4c2ba426cc01de5ce8adcdcae    11617
3a5cc5ac41ab3b18b63d6d8366e30fb0    11518
31c4eba2695cea9bec8abf0bf5

In [16]:
df_motion.location.value_counts()

location
kitchen         158415
lounge          153080
hallway          97011
bedroom1         75433
bathroom1        49507
dining room      17053
WC1              10723
conservatory      7715
study             5845
living room       5535
Name: count, dtype: int64

In [17]:
df_motion[['id', 'home_id', 'location']] = df_motion[['id', 'home_id', 'location']].astype('string')

In [18]:
df_motion.datetime = pd.to_datetime(df_motion.datetime, format='mixed', utc=False)

## Joining the tables

In [19]:
df_join = df_homes.merge(df_motion, on='home_id')
df_join

,home_id,multiple_occupancy,id,datetime,location
0,16d71b9c46d9abd765bf395818efe527,0,d68b171e348758a47d016f1564532dfe,2024-01-01 10:53:48+00:00,lounge
1,16d71b9c46d9abd765bf395818efe527,0,a7c2c8dd668e34d559797f4efecc1e2b,2024-01-01 10:55:40+00:00,lounge
2,16d71b9c46d9abd765bf395818efe527,0,07171a04ae1af97b1c6d11a1dc3ac89f,2024-01-01 10:56:47+00:00,lounge
3,16d71b9c46d9abd765bf395818efe527,0,fe6510edcd2ca6d95183a554e598a7d2,2024-01-01 10:57:12+00:00,lounge
4,16d71b9c46d9abd765bf395818efe527,0,c1d5917ca6f695ea971b8251ef1d80e0,2024-01-01 10:59:31+00:00,lounge
...,...,...,...,...,...
580312,3a1f53e2e242bff03301a250135f894b,0,b1e1ae91265ffd838686669e1bc3b8dd,2024-01-31 23:54:14+00:00,bedroom1
580313,3a1f53e2e242bff03301a250135f894b,0,18ac0583c2216bf23fb58505e999865f,2024-01-31 23:55:08+00:00,bedroom1
580314,3a1f53e2e242bff03301a250135f894b,0,54b172e8d4c07245901e38af65263efc,2024-01-31 23:55:38+00:00,bathroom1
580315,3a1f53e2e242bff03301a250135f894b,0,5ec75a93dcc136e6a4c7a4483d60b2ed,2024-01-31 23:58:26+00:00,bathroom1


## Extracting features from datetime 

In [20]:
df_join['year'] = df_join['datetime'].dt.year
df_join['month'] = df_join['datetime'].dt.month
df_join['day'] = df_join['datetime'].dt.day
df_join['hour'] = df_join['datetime'].dt.hour
df_join['minute'] = df_join['datetime'].dt.minute
df_join

,home_id,multiple_occupancy,id,datetime,location,year,month,day,hour,minute
0,16d71b9c46d9abd765bf395818efe527,0,d68b171e348758a47d016f1564532dfe,2024-01-01 10:53:48+00:00,lounge,2024,1,1,10,53
1,16d71b9c46d9abd765bf395818efe527,0,a7c2c8dd668e34d559797f4efecc1e2b,2024-01-01 10:55:40+00:00,lounge,2024,1,1,10,55
2,16d71b9c46d9abd765bf395818efe527,0,07171a04ae1af97b1c6d11a1dc3ac89f,2024-01-01 10:56:47+00:00,lounge,2024,1,1,10,56
3,16d71b9c46d9abd765bf395818efe527,0,fe6510edcd2ca6d95183a554e598a7d2,2024-01-01 10:57:12+00:00,lounge,2024,1,1,10,57
4,16d71b9c46d9abd765bf395818efe527,0,c1d5917ca6f695ea971b8251ef1d80e0,2024-01-01 10:59:31+00:00,lounge,2024,1,1,10,59
...,...,...,...,...,...,...,...,...,...,...
580312,3a1f53e2e242bff03301a250135f894b,0,b1e1ae91265ffd838686669e1bc3b8dd,2024-01-31 23:54:14+00:00,bedroom1,2024,1,31,23,54
580313,3a1f53e2e242bff03301a250135f894b,0,18ac0583c2216bf23fb58505e999865f,2024-01-31 23:55:08+00:00,bedroom1,2024,1,31,23,55
580314,3a1f53e2e242bff03301a250135f894b,0,54b172e8d4c07245901e38af65263efc,2024-01-31 23:55:38+00:00,bathroom1,2024,1,31,23,55
580315,3a1f53e2e242bff03301a250135f894b,0,5ec75a93dcc136e6a4c7a4483d60b2ed,2024-01-31 23:58:26+00:00,bathroom1,2024,1,31,23,58


## Model training and prediction



In [21]:
data = df_join 

# Split data into features and target variable
X = data[['year', 'month', 'day', 'hour', 'minute', 'location']] 
y = data['multiple_occupancy'] 

# One-hot encode 'location' feature
X = pd.get_dummies(X, columns=['location'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6349341742486904
